In [1]:
%pip install transformers faiss-cpu numpy torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 83.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 63.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.4/436.4 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.8/792.8 kB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 435.0/435.0 kB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 81.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 14.3 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import faiss
import torch
from transformers import AutoTokenizer, AutoModel

In [2]:
# Step 1: Load the LLM
model_name = "distilbert-base-uncased"  # You can use any compatible model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

/media/nas/uwork1/shollbor/pys/lib64/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [3]:
# Step 2: Prepare some documents for the vector database
documents = [
    "The cat sat on the mat.",
    "The dog chased the ball.",
    "Birds fly in the sky.",
    "Fish swim in the ocean.",
    "Tables have four legs."
]

In [5]:
# Step 3: Encode documents into vectors
def encode_documents(documents):
    inputs = tokenizer(documents, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state.mean(dim=1)  # Average pooling
    return embeddings.numpy()

# Create the vector database
document_vectors = encode_documents(documents)
dim = document_vectors.shape[1]

In [6]:
# Step 4: Build the FAISS index
index = faiss.IndexFlatL2(dim)  # Using L2 distance
index.add(document_vectors)  # Add document vectors to the index

In [7]:
# Step 5: Define a function for RAG
def retrieve_and_generate(query):
    # Encode the query
    query_vector = encode_documents([query])

    # Retrieve top-k similar documents
    k = 1  # Number of top results to retrieve
    D, I = index.search(query_vector, k)  # D: distances, I: indices

    # Get the relevant documents
    relevant_docs = [documents[i] for i in I[0]]

    # Simple "generation" (for demonstration, just concatenate)
    response = " ".join(relevant_docs)
    return response

In [8]:
# Step 6: Use the RAG system
query = "What do animals do?"
response = retrieve_and_generate(query)
print("Response:", response)

Response: Fish swim in the ocean.


In [9]:
query = "What do you know about barking "
response = retrieve_and_generate(query)
print("Response:", response)

Response: The dog chased the ball.
